# Module 2:
Make API call
- Collect data from API &amp; store in csv
- Scrape static websites &amp; store in csv
---
Scenario
---

1. You have been given a API with api documentation.
2. Make api call from python & store the resp in a python dict
2. (2A) *Optional*: Store the resp dict into a file (json)

3. Parse the following info from the API Response.

Example: URL 

https://min-api.cryptocompare.com/data/price?fsym=USD&tsyms=JPY,INR
```
{
"JPY": 107.93,
"INR": 84.82
}
```
4. Store that parsed infomation/data into a csv file.

| sno | from_symbol | to_symbol | price | datetime |
| ---  | ---  | ---  | ---  | --- |
| 1 | USD | INR | 71 | ... |
| 2 | USD | SGD | 1.37| ... |

use python `time` or `datetime` module to get the current time, when you make api call and store that into the csv file..


## Part 2: Scrape static websites


https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=SGD


![https://i.imgur.com/C4Eub9d.png](https://i.imgur.com/C4Eub9d.png)



Have a look at the html as well, to parse the exchange rate.

![https://i.imgur.com/vDAntcv.png](https://i.imgur.com/vDAntcv.png)

1. Look at the query parameter and decide how to pass inputs..
2. Use `bs4` library to parse the html as show below.
3. You have to extract the price shown here and save it into a csv file..

| sno | from_symbol | to_symbol | price | datetime |
| ---  | ---  | ---  | ---  | --- |
| 1 | USD | SGD | 1.37| ... |

--- 


** Try this if you completed the base workshop **
---
1. Store the same data into mongodb as well
  - create a `exchange` db in your mlab account.
  - create a collection `exchange-rates`
  - You can store sample record which looks like the following json.

```
{
    "from_symbol": "USD",
    'to_symbol' : "SGD",
    "price" : 1.37,
    "datetime" : "...."
}
```

# Part 1

In [127]:
import requests  
import selenium  
import csv
import pandas as pd
import psutil
import sched, time
from datetime import datetime
from bs4 import BeautifulSoup

#ask for user inputs
currency_to_convert = input("please key in a currency you want to convert from: ")
no_of_currencies = int(input("please key in the number of currencies you would like to convert to: "))
currencies_to_convert_to = ""

for i in range(no_of_currencies):
    currency_symbol = input("please key in one symbol of the currency you wish to convert to: " )
    if(currencies_to_convert_to != ""):
        currencies_to_convert_to = currencies_to_convert_to + ","
    currencies_to_convert_to = currencies_to_convert_to + currency_symbol

#construct url based on user input
url =  f"https://min-api.cryptocompare.com/data/price?fsym={currency_to_convert}&tsyms={currencies_to_convert_to}"

#define function to query website
def get_html_data(url: str)->str: # from python 3.7 onwards can use this line to specify the return type to be string
    resp = requests.get(url)
    htmldata = resp.text
    return htmldata
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36'
}
resp = requests.get(url )

#store resp in a python dict
data = resp.json()

#print response received
print(data)

#store time of query 
currentTime = datetime.now()

#Store resp dict in a file
my_file=open(r"QueryResponse.txt","w") 
my_file.write(str(data))
my_file.close()

#declare filename for part 1
fname = "TabulatedQueryResults.csv"

#store parsed info into a csv file
def save_metrics(row,mode):
    f= open(fname, mode=mode)
    f.write(row)
    f.close
            
#write headers into csv
header_row = f"sno,from_symbol,to_symbol,price,datetime\n"
save_metrics(header_row,mode = "w")
counter = 1

#write each result into csv
for item in list(data.items()):
    row = f"{counter},{str(currency_to_convert)},{item[0]},{item[1]},{datetime.strftime(currentTime, '%Y / %m / %d %H:%M:%S')}\n"
    save_metrics(row,mode = "a")
    counter= counter+1


please key in a currency you want to convert from: SGD
please key in the number of currencies you would like to convert to: 2
please key in one symbol of the currency you wish to convert to: USD
please key in one symbol of the currency you wish to convert to: SGD
{'USD': 0.7266, 'SGD': 1}


# Part 2 

In [125]:
import requests 
import selenium 
from bs4 import BeautifulSoup 
from functools import partial
from selenium import webdriver


#filename for part 2
fname2 = "results_website_scraping.csv"

#declare function to write into csv
def save_metrics(row,mode):
    f= open(fname2, mode=mode)
    f.write(row)
    f.close


#Ask for user inputs
number_of_query = int(input("please key in the number of conversions you would like to perform :"))

query_no = 1

#write in headers for csv file
header_row = f"sno,from_symbol,to_symbol,price,datetime\n"
save_metrics(header_row,mode = "w")
    
while query_no<=number_of_query:
    original_currency = input("please key in a currency you want to convert from: ")
    converted_currency = input("please key in the currency you would like to convert to: ")
    amount_to_convert = int(input(f"please key in the amount of {original_currency} that you would like to convert : "))

    print("please wait while we process your query...")
 
    #construct url
    url = f"https://www.xe.com/currencyconverter/convert/?Amount={amount_to_convert}&From={original_currency}&To={converted_currency}" 

    #store time of query 
    currentTime = datetime.now()

    #Read website data
    headers = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36'
    }
    driver = webdriver.Chrome(r"C:\Users\currypuff\Downloads\chromedriver_win32 (3)\chromedriver.exe")
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit() 

    #extract interesting portions
    h1_tags = soup.find_all("h1")  
    result = str(h1_tags)  
    
    words = result.split(converted_currency)
   
    conversion_results = words[1].split(" ") 

    #write results to csv
    row=f"{query_no},{original_currency},{converted_currency},{conversion_results[2]},{datetime.strftime(currentTime, '%Y / %m / %d %H:%M:%S')}\n"
    save_metrics(row,mode = "a")
    
    query_no +=1

print("Query complete. You may now open the excel file")

please key in the number of conversions you would like to perform :3
please key in a currency you want to convert from: SGD
please key in the currency you would like to convert to: JPY
please key in the amount of SGD that you would like to convert : 2
please wait while we process your query...
please key in a currency you want to convert from: JPY
please key in the currency you would like to convert to: USD
please key in the amount of JPY that you would like to convert : 390
please wait while we process your query...
please key in a currency you want to convert from: SGD
please key in the currency you would like to convert to: USD
please key in the amount of SGD that you would like to convert : 33
please wait while we process your query...
Query complete. You may now open the excel file
